# Challenge Set 1: MTA Turnstile Data

Problem statement:
WTWY needs to optimize their street teams so that they can get the most email signups for their annual OMGYN Gala. Ideally, the email signups will convert at a high rate to gala attendees, and among attendees, many will make contributions to WTWY.



Objective: Task to find supplemental data to incorporate into MTA trunstile data. 

Methodology: Use the geographic coordinates of the subway station map it to the US Census using the US Census Geocoder, to match the corresponding census tract GEOID.  Using the NYC Census finder to download demographic, social, economic and housing info for all New York Counties

In [2]:
import pandas as pd
from datetime import datetime, timedelta
import pickle

In [3]:
#Import CSV files, data from NYC Census Finder for all of New York Counties
def get_census_data(file_url):
    data = pd.read_csv(file_url, skiprows = 0)
    return data

census_data = get_census_data('ACS_17_5YR_S2301_with_ann.csv')
census_data = census_data.reindex(census_data.index.drop(0)).reset_index(drop=True)
census_data.rename(index=str, columns={"GEO.id2": "GEOID"},inplace=True)

In [4]:
census_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 288 entries, 0 to 287
Columns: 283 entries, GEO.id to HC04_MOE_VC47
dtypes: object(283)
memory usage: 639.0+ KB


In [5]:
census_data.head(5)

,GEO.id,GEOID,GEO.display-label,HC01_EST_VC01,HC01_MOE_VC01,HC02_EST_VC01,HC02_MOE_VC01,HC03_EST_VC01,HC03_MOE_VC01,HC04_EST_VC01,...,HC04_EST_VC46,HC04_MOE_VC46,HC01_EST_VC47,HC01_MOE_VC47,HC02_EST_VC47,HC02_MOE_VC47,HC03_EST_VC47,HC03_MOE_VC47,HC04_EST_VC47,HC04_MOE_VC47
0,1400000US36061000100,36061000100,"Census Tract 1, New York County, New York",0,11,-,**,-,**,-,...,-,**,0,11,-,**,-,**,-,**
1,1400000US36061000201,36061000201,"Census Tract 2.01, New York County, New York",2187,318,46.4,7.8,45.1,8,2.7,...,0.8,2.8,160,68,81.3,16.3,77.5,17.4,4.6,7.7
2,1400000US36061000202,36061000202,"Census Tract 2.02, New York County, New York",6524,523,49.5,5.4,41.2,5.6,16.8,...,22.3,18.1,1325,279,86.4,6.9,84,8.2,2.8,3.1
3,1400000US36061000500,36061000500,"Census Tract 5, New York County, New York",0,11,-,**,-,**,-,...,-,**,0,11,-,**,-,**,-,**
4,1400000US36061000600,36061000600,"Census Tract 6, New York County, New York",9607,1014,42,5.9,36.7,5.8,12.5,...,0,5.3,1302,471,81.9,12.2,66.9,14.8,18.3,17.6


In [6]:
#Download csv from the MTA website a list of subway stations with corresponding geo coordinates
def get_subway_station(file_url):
    data = pd.read_csv(file_url)
    return data

subway_station = get_subway_station('subway_station.csv')

In [312]:
subway_station.head()

,URL,OBJECTID,NAME,the_geom,LINE,NOTES
0,http://web.mta.info/nyct/service/,1,Astor Pl,POINT (-73.99106999861966 40.73005400028978),4-6-6 Express,"4 nights, 6-all times, 6 Express-weekdays AM s..."
1,http://web.mta.info/nyct/service/,2,Canal St,POINT (-74.00019299927328 40.71880300107709),4-6-6 Express,"4 nights, 6-all times, 6 Express-weekdays AM s..."
2,http://web.mta.info/nyct/service/,3,50th St,POINT (-73.98384899986625 40.76172799961419),1-2,"1-all times, 2-nights"
3,http://web.mta.info/nyct/service/,4,Bergen St,POINT (-73.97499915116808 40.68086213682956),2-3-4,"4-nights, 3-all other times, 2-all times"
4,http://web.mta.info/nyct/service/,5,Pennsylvania Ave,POINT (-73.89488591154061 40.66471445143568),3-4,"4-nights, 3-all other times"


In [7]:
#Need to do a bit of data cleaning on the_geom to break out the coordinates and convert it to a string list
subway_station['the_geom'] = subway_station['the_geom'].str.replace("POINT","")
subway_station['the_geom'] = subway_station['the_geom'].str.replace("(","")
subway_station['the_geom'] = subway_station['the_geom'].str.replace(")","")
subway_station['the_geom'] = subway_station['the_geom'].str.lstrip()
subway_station['the_geom'] = subway_station['the_geom'].str.split(" ")
subway_station.head(5)

,URL,OBJECTID,NAME,the_geom,LINE,NOTES
0,http://web.mta.info/nyct/service/,1,Astor Pl,"[-73.99106999861966, 40.73005400028978]",4-6-6 Express,"4 nights, 6-all times, 6 Express-weekdays AM s..."
1,http://web.mta.info/nyct/service/,2,Canal St,"[-74.00019299927328, 40.71880300107709]",4-6-6 Express,"4 nights, 6-all times, 6 Express-weekdays AM s..."
2,http://web.mta.info/nyct/service/,3,50th St,"[-73.98384899986625, 40.76172799961419]",1-2,"1-all times, 2-nights"
3,http://web.mta.info/nyct/service/,4,Bergen St,"[-73.97499915116808, 40.68086213682956]",2-3-4,"4-nights, 3-all other times, 2-all times"
4,http://web.mta.info/nyct/service/,5,Pennsylvania Ave,"[-73.89488591154061, 40.66471445143568]",3-4,"4-nights, 3-all other times"


In [8]:
#Create two new columns for Longitude and Latitude
subway_station.loc[:,'LAT'] = subway_station['the_geom'].map(lambda x: x[0])
subway_station.loc[:,'LONG'] = subway_station['the_geom'].map(lambda x: x[1])
subway_station.head(5)

,URL,OBJECTID,NAME,the_geom,LINE,NOTES,LAT,LONG
0,http://web.mta.info/nyct/service/,1,Astor Pl,"[-73.99106999861966, 40.73005400028978]",4-6-6 Express,"4 nights, 6-all times, 6 Express-weekdays AM s...",-73.99106999861966,40.73005400028978
1,http://web.mta.info/nyct/service/,2,Canal St,"[-74.00019299927328, 40.71880300107709]",4-6-6 Express,"4 nights, 6-all times, 6 Express-weekdays AM s...",-74.00019299927328,40.71880300107709
2,http://web.mta.info/nyct/service/,3,50th St,"[-73.98384899986625, 40.76172799961419]",1-2,"1-all times, 2-nights",-73.98384899986625,40.76172799961419
3,http://web.mta.info/nyct/service/,4,Bergen St,"[-73.97499915116808, 40.68086213682956]",2-3-4,"4-nights, 3-all other times, 2-all times",-73.97499915116808,40.68086213682956
4,http://web.mta.info/nyct/service/,5,Pennsylvania Ave,"[-73.89488591154061, 40.66471445143568]",3-4,"4-nights, 3-all other times",-73.89488591154061,40.66471445143568


In [9]:
#Found a censusgeo code modle that is a light weight Python wrapper for US Census Geocoder API
#First time use make sure you install censusgeocode 
#!pip install censusgeocode
import censusgeocode as cg

def get_geoid_coord(lat,long):
    """ Utilizes Census Geocode which is a light weight Python wrapper for the US Census Geocoder API
        tool for geocoding an address to a longitude and latitude, 
        or a batch file into a parsed address and coordinates. 
        
        input: latitude and longitude
        output: corresponding census tracts
        """
    results = cg.coordinates(x=lat, y=long)
    
#Get the corresponding census tract code GEOID given the coordinates
    return results['Census Tracts'][0]["GEOID"]

In [10]:
#Test to see given a set of coordinates does it return the correct GEOID
censustract =  get_geoid_coord(-74.00858473570133, 40.714111000774025)
censustract

'36061002100'

In [426]:
#Create a new column in subway_station dataframe and apply the function for each row, passing the LAT and LON
subway_station['GEOID'] = subway_station[['LAT','LONG']].apply(lambda x:get_geoid_coord(x.LAT,x.LONG), axis=1)

In [427]:
#Pull up one example and check that it pulled the up correct geocode
subway_station.iloc[120]

URL                http://web.mta.info/nyct/service/
OBJECTID                                         121
NAME                  Atlantic Av - Barclay's Center
the_geom    [-73.97678343963167, 40.684488323453685]
LINE                                             B-Q
NOTES           B-weekdays and evenings, Q-all times
LAT                               -73.97678343963167
LONG                              40.684488323453685
GEOID                                    36047003500
Name: 120, dtype: object

In [429]:
#Saving my dataframe with the new data into a pickle and reopening it 
with open('subway_station.pickle', 'wb') as to_write:
    pickle.dump(subway_station, to_write)

In [430]:
with open('subway_station.pickle','rb') as read_file:
    subway_station = pickle.load(read_file)

In [442]:
#Merge the subway_station dataframe with our census data on GEOID to create a combined dataframe
subway_census = pd.merge(subway_station, census_data, on='GEOID')

In [443]:
subway_census.head(5)

,URL,OBJECTID,NAME,the_geom,LINE,NOTES,LAT,LONG,GEOID,GEO.id,...,HC04_EST_VC46,HC04_MOE_VC46,HC01_EST_VC47,HC01_MOE_VC47,HC02_EST_VC47,HC02_MOE_VC47,HC03_EST_VC47,HC03_MOE_VC47,HC04_EST_VC47,HC04_MOE_VC47
0,http://web.mta.info/nyct/service/,1,Astor Pl,"[-73.99106999861966, 40.73005400028978]",4-6-6 Express,"4 nights, 6-all times, 6 Express-weekdays AM s...",-73.99106999861966,40.73005400028978,36061005700,1400000US36061005700,...,0,46.8,1650,204,89.4,3.8,87.9,3.9,1.2,1.3
1,http://web.mta.info/nyct/service/,401,8th St - NYU,"[-73.99250799849149, 40.73046499853991]",N-Q-R-W,"N-all times, Q-nights, R-all times exc nights",-73.99250799849149,40.73046499853991,36061005700,1400000US36061005700,...,0,46.8,1650,204,89.4,3.8,87.9,3.9,1.2,1.3
2,http://web.mta.info/nyct/service/,2,Canal St,"[-74.00019299927328, 40.71880300107709]",4-6-6 Express,"4 nights, 6-all times, 6 Express-weekdays AM s...",-74.00019299927328,40.71880300107709,36061004500,1400000US36061004500,...,16.2,21.8,485,101,84.5,6.1,81,6.4,4.1,3.6
3,http://web.mta.info/nyct/service/,417,Canal St,"[-74.0018260000577, 40.71946500105898]",R-W,"N-nights, R-all other times",-74.0018260000577,40.71946500105898,36061004500,1400000US36061004500,...,16.2,21.8,485,101,84.5,6.1,81,6.4,4.1,3.6
4,http://web.mta.info/nyct/service/,435,Canal St,"[-74.00105471306033, 40.718814263587134]",N-Q,"N-all times, Q-weekdays and evenings",-74.00105471306033,40.718814263587134,36061004500,1400000US36061004500,...,16.2,21.8,485,101,84.5,6.1,81,6.4,4.1,3.6


In [447]:
#Pickle the new dataframe
with open('subway_census.pickle', 'wb') as to_write:
    pickle.dump(subway_census, to_write)

In [459]:
#Note some of the columns have cryptic column names
subway_census.columns

Index(['URL', 'OBJECTID', 'NAME', 'the_geom', 'LINE', 'NOTES', 'LAT', 'LONG',
       'GEOID', 'GEO.id',
       ...
       'HC04_EST_VC46', 'HC04_MOE_VC46', 'HC01_EST_VC47', 'HC01_MOE_VC47',
       'HC02_EST_VC47', 'HC02_MOE_VC47', 'HC03_EST_VC47', 'HC03_MOE_VC47',
       'HC04_EST_VC47', 'HC04_MOE_VC47'],
      dtype='object', length=291)

### Columns to pull from the census ACS metadata with description
-------
#HC01_EST_VC29 - Total; Estimate; Population 20 to 64 years - SEX - Female
#HC02_EST_VC01 - Labor Force Participation Rate; Estimate; Population 16 years and over
#HC02_EST_VC29 - Labor Force Participation Rate; Estimate; Population 20 to 64 years - SEX - Female
#HC04_EST_VC29 - Unemployment rate; Estimate; Population 20 to 64 years - SEX - Female
#HC02_EST_VC47 - Total; Estimate; EDUCATIONAL ATTAINMENT - Population 25 to 64 years - Bachelor's degree or higher

In [585]:
#create a demographic data from with the columns that we want
subway_demog = subway_census[['NAME',
                              'HC02_EST_VC01',
                              'HC02_EST_VC29',
                              'HC04_EST_VC29',
                              'HC01_EST_VC43' ]]

In [569]:
#change the column names to something more decriptive
subway_demog.columns = ['Subway Station',
                        'Labor Force Participation',
                        'Female Labor Force Participation',
                        'Female Unemployment Rate',
                        'Bachelor degree or higher']

In [570]:
#Note there are duplicates, keep the last
subway_demog_clean = subway_demog.drop_duplicates(subset='Subway Station', keep="last

In [590]:
#To get statistics we need to convert the columns to integers
subway_demog_clean['Female Labor Force Participation'] = pd.to_numeric(subway_demog_clean['Female Labor Force Participation'])
subway_demog_clean['Labor Force Participation'] = pd.to_numeric(subway_demog_clean['Labor Force Participation'])

In [591]:
subway_demog_clean.head(5)

,Subway Station,Labor Force Participation,Female Labor Force Participation,Female Unemployment Rate,Bachelor degree or higher
0,Astor Pl,69.9,83.4,0.9,1759
1,8th St - NYU,69.9,83.4,0.9,1759
7,7th Ave,76.5,78.0,4,1929
10,Delancey St - Essex St,64.9,77.4,7.6,4922
11,Grand St,64.9,77.4,7.6,4922


In [592]:
#Import our top 10 stations via a list, note there was slight discrepancies in the naming conventions
top_10 = ['34th St - Penn Station', 
          'Grand Central - 42nd St', 
          'Fulton St',
          '23rd St',
          'Wall St',
          '5th Ave - 53rd St',
        '47th-50th Sts - Rockefeller Ctr',
        'Lexington Ave - 53rd St',
          'Chambers St',
          '34th St - Hudson Yards']

In [593]:
#Filter the dataframe for only our top 10 stations
top10_df = subway_demog_clean[subway_demog_clean['Subway Station'].isin(top_10)]

In [594]:
top10_df

,Subway Station,Labor Force Participation,Female Labor Force Participation,Female Unemployment Rate,Bachelor degree or higher
50,5th Ave - 53rd St,83.3,76.3,0,87
51,Lexington Ave - 53rd St,77.6,91.0,2.1,4713
80,Grand Central - 42nd St,74.7,85.1,2.3,3095
94,47th-50th Sts - Rockefeller Ctr,88.7,70.8,0,112
106,34th St - Penn Station,85.1,72.0,3.4,127
109,23rd St,76.3,78.0,8.3,2328
120,Fulton St,80.8,79.9,4.3,5235
125,Chambers St,42.7,68.4,4.5,4178
136,Wall St,88.6,90.9,4.6,6208
148,34th St - Hudson Yards,86.2,81.9,3.8,4694


In [595]:
#Convert the other 2 columns to numeric float 64
top10_df['Female Unemployment Rate'] = pd.to_numeric(top10_df['Female Unemployment Rate'])
top10_df['Bachelor degree or higher'] = pd.to_numeric(top10_df['Bachelor degree or higher'])

/Users/robfang29/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/robfang29/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [596]:
top10_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 50 to 148
Data columns (total 5 columns):
Subway Station                      10 non-null object
Labor Force Participation           10 non-null float64
Female Labor Force Participation    10 non-null float64
Female Unemployment Rate            10 non-null float64
Bachelor degree or higher           10 non-null int64
dtypes: float64(3), int64(1), object(1)
memory usage: 480.0+ bytes


In [597]:
#Sort our top 10 by Labor force participation and female labor force participation
top10_df.sort_values(by=['Labor Force Participation', 'Female Labor Force Participation'], ascending=[False,False] )

,Subway Station,Labor Force Participation,Female Labor Force Participation,Female Unemployment Rate,Bachelor degree or higher
94,47th-50th Sts - Rockefeller Ctr,88.7,70.8,0.0,112
136,Wall St,88.6,90.9,4.6,6208
148,34th St - Hudson Yards,86.2,81.9,3.8,4694
106,34th St - Penn Station,85.1,72.0,3.4,127
50,5th Ave - 53rd St,83.3,76.3,0.0,87
120,Fulton St,80.8,79.9,4.3,5235
51,Lexington Ave - 53rd St,77.6,91.0,2.1,4713
109,23rd St,76.3,78.0,8.3,2328
80,Grand Central - 42nd St,74.7,85.1,2.3,3095
125,Chambers St,42.7,68.4,4.5,4178


In [598]:
#Get statistics on our top 10 subway station
top10_df.describe()

,Labor Force Participation,Female Labor Force Participation,Female Unemployment Rate,Bachelor degree or higher
count,10.000000,10.000000,10.000000,10.000000
mean,78.400000,79.430000,3.330000,3077.700000
std,13.494279,7.937821,2.440423,2308.406283
min,42.700000,68.400000,0.000000,87.000000
25%,76.625000,73.075000,2.150000,677.250000
50%,82.050000,78.950000,3.600000,3636.500000
75%,85.925000,84.300000,4.450000,4708.250000
max,88.700000,91.000000,8.300000,6208.000000


###### On average these areas have high female labor force participation

In [600]:
#Corresponding statistics for the entire manhattan
subway_demog_clean.describe()

,Labor Force Participation,Female Labor Force Participation
count,94.000000,94.000000
mean,68.147872,74.175532
std,15.929676,14.456607
min,0.000000,0.000000
25%,64.450000,71.025000
50%,70.000000,77.550000
75%,76.950000,81.225000
max,90.800000,100.000000
